In [4]:
import cv2
import numpy as np

In [5]:
from pynput.mouse import Button, Controller

mouse = Controller()

In [7]:
cap = cv2.VideoCapture(0)
# buttonPressed = False
while True:
    _,frame = cap.read()
    
    
    
    frame1 = frame[:240,320:,]
    hsv = cv2.cvtColor(frame1,cv2.COLOR_BGR2HSV)
    lower_blue = np.array([110,100,100])
    upper_blue = np.array([130,255,255])
#     lower_red = np.array([170,120,70])
#     upper_red = np.array([180,255,255])
#     mask_red = cv2.inRange(hsv,lower_red,upper_red)
    mask = cv2.inRange(hsv,lower_blue,upper_blue)
#     blur_red = cv2.medianBlur(mask_red,11)
    blur = cv2.medianBlur(mask,11)

    cut = cv2.bitwise_and(frame1,frame1,mask=blur)
    
#     cont_red,heir = cv2.findContours(blur_red,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cont,heir = cv2.findContours(blur,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     print(len(cont))
    
    if len(cont) ==1:
        hull = cv2.convexHull(cont[0],False)
        fin = cv2.drawContours(cut,[hull],-1,(0,0,255),1)
        M = cv2.moments(hull)
#         if not(buttonPressed):
        mouse.press(Button.left)
#             buttonPressed =True
        if M["m00"]!=0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            mouse.position = ((cX/320)*1600,(cY/180)*800)
            fin = cv2.circle(fin,(cX,cY),3,(0,0,255),-1)
        frame[:240,320:,] = fin
    elif len(cont)==2:
#         if buttonPressed:
        mouse.release(Button.left)
#             buttonPressed =False
            
            
        hull1 = cv2.convexHull(cont[0],False)
        fin =cv2.drawContours(cut,[hull1],-1,(0,255,0),1)
        M1 =cv2.moments(hull1)
        flag1=False
        if M1["m00"]!=0.0:
            flag1=True
            c1X = int(M1["m10"] / M1["m00"])
            c1Y = int(M1["m01"] / M1["m00"])
            fin =cv2.circle(fin,(c1X,c1Y),3,(0,255,0),1)
        
        hull2 = cv2.convexHull(cont[1],False)
        fin =cv2.drawContours(fin,[hull2],-1,(0,0,255),1)
        M2 =cv2.moments(hull2)
        flag2=False
        if M1["m00"]!=0.0:
            flag2 =True
            c2X = int(M2["m10"] / M2["m00"])
            c2Y = int(M2["m01"] / M2["m00"])
            fin =cv2.circle(fin,(c2X,c2Y),3,(0,0,255),1)
            
        if flag1 and flag2:
            cX = int((c1X+c2X) /2)
            cY = int((c1Y+c2Y) /2)
            mouse.position = ((cX/320)*1600,(cY/180)*800)
        
        frame[:240,320:,] = fin
    else:
        frame[:240,320:,] = cut
    
        
    
    cv2.imshow('frame',frame)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# 480,640,3